In [17]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [18]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [19]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [20]:
SYSTEM_PROMPT = """
Based on the provided use case specification, please generate a set of test scenarios to thoroughly test the functionality described in the use case.
Your task is to generate suitable test scenarios that cover all the flows and test the boundaries of conditions, if possible. Each scenario should clearly and specifically describe the situation to be tested.
Example for condition: "a valid username must be over 8 characters and below 30 characters." When you meets something like this, make sure to treat each condition as a negative scenario seperately. That means you will have two negative scenario: invalid username with lower than 8 characters and invalid username with higher than 30 characters. 
Please ensure that your test scenarios cover both the main flow and any alternate flows mentioned in the use case specification. Also, consider potential exception flows to address errors, failures, or unexpected events that could occur during the execution of the use case and encompass negative test scenarios (including edge cases and invalid inputs), and unexpected user behaviors.
Keep in mind the agile nature of the application and prioritize scenarios that align closely with the given use case specification and the application's requirements.
Return the list of test scenarios name to test the use case
Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not arbitrarily create additional test scenarios to test cases that not mention in the use case flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

In [21]:
usecase_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Book(Github)"
# save_path = r"D:\Dissertation-GPT\evaluate\prove\Matcha\ts"

In [22]:
usecase_name_list = []
project_name = os.path.basename(usecase_path)

In [23]:
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)
# usecase_name_list=["AddQuestion "]

['Add a new author', 'Add a new book', 'Add a new genre', 'Display author details', 'Display book details', 'Display genre details', 'Display the list of authors', 'Display the list of books', 'Display the list of genres', 'Edit the author', 'Edit the book', 'Edit the genre', 'Remove the author', 'Remove the book', 'Remove the genre']


In [24]:
for usecase_name in usecase_name_list:
    print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    prompt_all = read_file_content(usecase_directlink)
    promptSYSTEM_PROMPT = [
        { "role": "system", "content": SYSTEM_PROMPT},
        { "role": "user", "content": prompt_all}
    ]
    gpt_response = ask(promptSYSTEM_PROMPT, client, model)    
    print("gpt_response \n" +gpt_response)
    # write_string_to_file(os.path.join(save_path,f"{usecase_name}.txt"), gpt_response)

Add a new author
gpt_response 
1. SuccessfulAuthorCreation
2. SubmitWithIncompleteData
3. SubmitWithInvalidData
4. DisplayErrorMessageOnDatabaseError
5. CancelOperationBeforeDataSubmission
6. NavigateAwayBeforeDataSubmission
7. SubmitWithBoundaryNameLength
8. SubmitWithBoundaryBioLength
9. SubmitWithMaximumNameLength
10. SubmitWithMaximumBioLength
11. SubmitWithMinimumNameLength
12. SubmitWithMinimumBioLength
Add a new book
gpt_response 
1. Successful book creation
2. Book creation with invalid title
3. Book creation with invalid author
4. Book creation with invalid genre
5. Book creation with invalid description
6. Book creation with invalid publication date
7. Book creation with database error
8. Cancel book creation before submitting
9. Navigate away from book creation form to "Books"
10. Navigate away from book creation form to "Genres"
11. Navigate away from book creation form to "Authors"
12. Navigate away from book creation form to "Home"
13. Navigate away from book creation for